# Branch and Bound for the Quadratic Assignment Problem

In [1]:
##This is an initialization cell. Run this first
import pandas as pd
import numpy as np
from itertools import product
import time
import math
import matplotlib
import matplotlib.pyplot as plt

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    
    Optimised 04/06/2020
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = np.zeros(column)
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = np.array([])
            for i in string2:
                if i != '':
                    emptyarray = np.append(emptyarray,i).astype(int)
            matrix = np.vstack((matrix,emptyarray))
        return matrix[1:]
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_3 = './data/made3.csv'
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = './data/tai10a.csv'
matrix_size_11 = './data/made11.csv'
matrix_size_12 = './data/tai12a.csv'
matrix_size_15 = './data/chr15a.csv' 
matrix_size_20 = './data/chr20a.csv'
matrix_size_26 = './data/bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = './data/tai40a.csv'
matrix_size_60 = './data/tai60.csv'
matrix_size_80 = './data/tai80.csv'
matrix_size_256 = './data/tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

Below is the permutation generating function. For medium and large instances it would be wise to run this function to file separately first and then use the results for Exhaustive Search and/or Branch and Bound.

In [4]:
def QAPPermutations(iterable, r=None):
    """
    Input:
    String or numbers separated by a space
    optional= the length that the permutations must be
    
    Output:
    a generator of permutations
    
    Sourced from the itertools library and adjusted to suit this data type
    https://docs.python.org/3.4/library/itertools.html#itertools.permutations 
    """
    
    pool = iterable.split(" ")
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield list(pool[i] for i in indices)

In [5]:
def BranchandBound(MatrixLoc,MatrixFlow):
    """
    Input:
    MatrixLoc
    MatrixFlow
    
    Output:
    The optimal permutation
    the optimal cost
    in a tuple
    """
    #initialization
    start = time.time()
    matrix_length = len(MatrixLoc)
    arraysol = []
    bettersol = math.inf
    bettersolind = 0
    thestring = ""
    
    #Generate the permutations
    for i in range(0,matrix_length):  #this is making a string of numbers from 0 to the size of the matrix -1
        thestring += str(i) + " "
    thestring = thestring[:-1]
    listofpermutations = np.array(list(QAPPermutations(thestring,matrix_length))).astype(int)
    no_of_permutations = len(listofpermutations)
    
    #Generate the multiples (that function we are optimising)
    k = 0
    while k < no_of_permutations:
        perm = listofpermutations[k]
        total = 0
        i =0
        #bounding
        while i < matrix_length and bettersol>=total:
            j=0
            while j<matrix_length and bettersol>=total:
                if i!=j:
                    total += MatrixLoc[i][j]*MatrixFlow[perm[i]][perm[j]]
                j += 1
            i += 1
        #branching
        if i <= matrix_length - 2:
            temp = k
            temp += 1
            temperm = perm[:i]
            while temp<no_of_permutations and list(listofpermutations[temp][:i]) == list(temperm):
                temp += 1  
            k = temp - 1
        #retain best solution
        if bettersol>total:
            bettersol = total
            bettersolind = k
        k+=1
    
    finalcost = bettersol
    finalindex = bettersolind
    end = time.time()
    thetime = end - start
    
    return listofpermutations[finalindex],finalcost,thetime

In [6]:
datamatrix = CSVtoNumpyArray(matrix_size_3) # Decide the size of problem to run in the code 
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

print(BranchandBound(MatrixLoc,MatrixFlow))

(array([1, 0, 2]), 10216.0, 0.0)


In [12]:
#names = ["made4.csv","made5.csv","made6.csv","made7.csv","made8.csv","made9.csv","made10.csv","made11.csv"]
def BnBResults(names):
    file = open("BnB - out.csv","w")
    for q in range(len(names)):
        datamatrix = CSVtoNumpyArray(names[q]) # Decide the size of problem to run in the code
        MatrixLoc = datamatrix[0]
        MatrixFlow = datamatrix[1]
        sol = (q+4,BranchandBound(MatrixLoc,MatrixFlow)[-1])
        print(names[q] + " has been solved by BnB")
        file.write( str(sol[0])+  " " + str(sol[1])+ "\n")
    file.close()

    return True